<a href="https://colab.research.google.com/github/Fuenfgeld/Agent_Tutorial_PydanticAI/blob/main/04_Context_Memory__DependencyInjection_PydanticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install pydantic-ai
%pip -q install nest_asyncio
%pip -q install logfire

In [ ]:
import os
from google.colab import userdata

keyAntropic = userdata.get('Claude')
keyOpenAI = userdata.get('openAI')
keyLogFire = userdata.get('logfire')


os.environ["OPENAI_API_KEY"] = keyOpenAI
os.environ["ANTHROPIC_API_KEY"] = keyAntropic

import nest_asyncio
nest_asyncio.apply()
#logfire.configure(token=keyLogFire)

## Dependency Injection
are a way of giving Run Context specific information that can be used in system Prompts, tools, validators.


In [ ]:
import os
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel


# This example demonstrates how to use dependency injection in Pydantic AI to provide personalized healthcare career transition advice.
# The system prompt uses the healthcare specialty provided by the user to tailor recommendations for integrating AI into their career path.

# Define the model
model = OpenAIModel('gpt-4o-mini')

# Define the agent with a system prompt
agent = Agent(
    model=model,
    system_prompt = """You are an experienced healthcare career advisor specializing in digital health transformation.
                 Guide healthcare professionals as they adapt their careers to incorporate AI and digital technologies. """,

    deps_type=str
)

# Define a system prompt with dependency injection
@agent.system_prompt
def get_healthcare_specialty(ctx: RunContext[str]) -> str:
    return f"The user's healthcare specialty is {ctx.deps}. Tailor your advice to this specific medical field."

# Define the main loop
def main_loop():
    print("===== Healthcare AI Career Advisor =====")
    print("This tool helps healthcare professionals adapt their careers to incorporate AI technologies.")

    while True:
        user_input = input("\n>> Please specify your healthcare specialty (e.g., radiology, nursing, pharmacy) or type 'exit' to quit: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Thank you for using the Healthcare AI Career Advisor. Goodbye!")
            break

        # Run the agent
        result = agent.run_sync("Provide a 10-step plan for integrating AI into my healthcare career.", deps=user_input)
        print(f"\nYour AI Healthcare Career Transition Plan:\n{result.output}")

# Run the main loop
if __name__ == "__main__":
    main_loop()

## Dependencies as Data Containers
Depenencies can be nested Pydantic Data classes that are declared during the Agent initialization and are given during runtime

## System Prompt Dependency Injection

In [ ]:
import os

from colorama import Fore
from pydantic_ai.models.openai import OpenAIModel
from pydantic import BaseModel
from dataclasses import dataclass
from pydantic_ai.agent import Agent, RunContext
from pydantic_ai.exceptions import ModelRetry


# This example demonstrates a medical assistant chatbot that helps patients with their
# electronic health record (EHR) information, medication questions, and appointment scheduling.

# Define the model
model = OpenAIModel('gpt-4o-mini')

class Medication(BaseModel):
    """Medication model - includes medication name, dosage, frequency and instructions"""
    medication_name: str
    dosage: str
    frequency: str
    instructions: str
    refills_remaining: int

# Define the output model
class Patient(BaseModel):
    """Patient model - includes patient ID, full name, date of birth and contact info"""
    patient_id: int
    name: str
    date_of_birth: str
    email: str
    phone: str
    medications: list[Medication]
    next_appointment: str

class MedicalAssistantResponse(BaseModel):
    """Medical assistant response model - includes patient info, response category,
    response text and whether a clinician needs to be consulted"""
    patient: Patient
    response_category: str  # medication, appointment, general info, clinical
    response: str
    clinician_consult_needed: bool

@dataclass
class Deps:
    """Dependencies for the agent"""
    patient: Patient

# Define the agent
agent = Agent(
    model=model,
    result_type=MedicalAssistantResponse,
    system_prompt="You are a medical assistant chatbot helping patients with their healthcare information. "
                 "Your task is to provide helpful information about their medications, appointments, "
                 "and general health record information. If the question requires clinical judgment "
                 "or is beyond your scope, indicate that a clinician needs to be consulted. "
                 "Always address the patient by name and be empathetic and professional. "
                 "End your response with 'Patient ID: [patient_id]'.",
    deps_type=Deps
)

@agent.system_prompt
def get_system_prompt(ctx: RunContext[Deps]) -> str:
    return f"The patient information is: {ctx.deps.patient}."

# Run the agent
try:
    # Create a patient profile with medications
    patient = Patient(
        patient_id=12345,
        name="Maria Garcia",
        date_of_birth="1985-06-15",
        email="maria.garcia@email.com",
        phone="555-123-4567",
        next_appointment="2023-11-15 10:30 AM",
        medications=[
            Medication(
                medication_name="Lisinopril",
                dosage="10mg",
                frequency="Once daily",
                instructions="Take in the morning with food",
                refills_remaining=2
            ),
            Medication(
                medication_name="Metformin",
                dosage="500mg",
                frequency="Twice daily",
                instructions="Take with meals",
                refills_remaining=3
            )
        ]
    )

    # Create dependencies
    deps = Deps(patient=patient)

    question = "What medications am I currently taking?"
    result = agent.run_sync(question, deps=deps)
    print(Fore.YELLOW, question)
    print(Fore.GREEN, f"Assistant: {result.output.response}")
    print(Fore.CYAN, f"Response category: {result.output.response_category}")
    print(Fore.RED, f"Clinician consult needed: {result.output.clinician_consult_needed}")
    print('\n-----------------------------------\n')

    question = "When is my next appointment scheduled?"
    result = agent.run_sync(question, deps=deps)
    print(Fore.YELLOW, question)
    print(Fore.GREEN, f"Assistant: {result.output.response}")
    print(Fore.CYAN, f"Response category: {result.output.response_category}")
    print(Fore.RED, f"Clinician consult needed: {result.output.clinician_consult_needed}")
    print('\n-----------------------------------\n')

    question = "I'm having side effects from my Metformin. What should I do?"
    result = agent.run_sync(question, deps=deps)
    print(Fore.YELLOW, question)
    print(Fore.GREEN, f"Assistant: {result.output.response}")
    print(Fore.CYAN, f"Response category: {result.output.response_category}")
    print(Fore.RED, f"Clinician consult needed: {result.output.clinician_consult_needed}")
    print('\n-----------------------------------\n')

    question = "Can you interpret my recent lab results and tell me if I need to change my diet?"
    result = agent.run_sync(question, deps=deps)
    print(Fore.YELLOW, question)
    print(Fore.GREEN, f"Assistant: {result.output.response}")
    print(Fore.CYAN, f"Response category: {result.output.response_category}")
    print(Fore.RED, f"Clinician consult needed: {result.output.clinician_consult_needed}")
    print('\n-----------------------------------\n')

except ModelRetry as e:
    print(Fore.RED, e)
except Exception as e:
    print(Fore.RED, f"Error: {e}")

## Dependency Injection in Tools

In [ ]:
import os

from colorama import Fore
from pydantic_ai import Agent, RunContext, ModelRetry
from pydantic_ai.models.openai import OpenAIModel
from pydantic import BaseModel
from dataclasses import dataclass

# This example demonstrates how to use tools with dependencies in PydanticAI.
# The use case is a healthcare authorization agent evaluating a patient's
# insurance coverage and approving or denying a treatment request.

# Define the model
model = OpenAIModel('gpt-4o-mini')

class InsurancePolicy(BaseModel):
    """Insurance policy model - includes policy details"""
    policy_number: int
    policy_type: str  # e.g., "HMO", "PPO", "Medicare"
    policy_status: str  # e.g., "Active", "Expired"
    annual_deductible: int
    out_of_pocket_max: int
    remaining_deductible: int
    requires_preauthorization: bool

# Define the patient model
class Patient(BaseModel):
    """Patient model - includes patient demographics and medical information"""
    patient_id: int
    name: str
    email: str
    age: int
    insurance_id: str
    diagnosis_code: str  # ICD-10 code
    procedure_code: str  # CPT code
    treatment_cost: int
    treatment_description: str
    provider_in_network: bool
    treatment_urgency: str  # e.g., "Elective", "Urgent", "Emergency"
    previous_treatments: str
    medical_history: str
    additional_notes: str

class AuthorizationDecision(BaseModel):
    """Authorization decision model - includes patient, decision type, explanation, and coverage details"""
    patient: Patient
    decision_type: str  # "Approved", "Denied", "Pending Additional Information"
    explanation: str
    covered_amount: int
    patient_responsibility: int
    authorization_code: str

@dataclass
class Deps:
    """Dependencies for the agent"""
    patient: Patient
    policy: InsurancePolicy

# Define the agent
agent = Agent(
    model=model,
    result_type=AuthorizationDecision,
    system_prompt="You are a healthcare insurance authorization specialist. Your task is to evaluate the patient's insurance coverage, analyze the treatment request, and determine if the requested procedure should be authorized. Provide a clear and helpful response. Ensure that your response is professional and addresses the patient's needs effectively. Always include the patient's name in your response. End your answer with Ref: Authorization Request ID.",
    deps_type=Deps
)

@agent.tool(retries=2)
async def get_coverage_analysis(ctx: RunContext[Deps]) -> str:
    """Analyze the patient's insurance coverage for the requested treatment."""

    # Call the llm to get the coverage analysis
    analysis_agent = Agent(
        model=model,
        result_type=str,
        system_prompt="Analyze the patient's insurance coverage for the requested treatment. Consider diagnosis codes, procedure codes, network status, and policy details.",
        deps_type=Deps
    )

    @analysis_agent.system_prompt
    def get_system_prompt(ctx: RunContext[str]) -> str:
        return f"Patient information: {ctx.deps.patient}\nInsurance policy: {ctx.deps.policy}"

    result = await analysis_agent.run("Provide a detailed coverage analysis for this treatment request.", deps=ctx.deps)
    print(Fore.BLUE, f"Coverage analysis: {result.output}")
    return f"Coverage analysis: {result.output}"


# Run the agent
try:
    # Create a patient profile and insurance policy
    patient = Patient(
        patient_id=12345,
        name="Sarah Johnson",
        email="sarah.j@example.com",
        age=45,
        insurance_id="INS789012",
        diagnosis_code="J45.901",  # Asthma, unspecified
        procedure_code="94640",    # Airway inhalation treatment
        treatment_cost=850,
        treatment_description="Nebulizer treatment for asthma exacerbation",
        provider_in_network=True,
        treatment_urgency="Urgent",
        previous_treatments="Albuterol inhaler, oral steroids",
        medical_history="Asthma (10 years), Seasonal allergies",
        additional_notes="Patient experienced severe asthma attack yesterday"
    )

    policy = InsurancePolicy(
        policy_number=56789,
        policy_type="PPO",
        policy_status="Active",
        annual_deductible=1500,
        out_of_pocket_max=5000,
        remaining_deductible=500,
        requires_preauthorization=True
    )

    # Create dependencies
    deps = Deps(patient=patient, policy=policy)

    question = "Please evaluate this treatment authorization request."
    result = agent.run_sync(question, deps=deps)

    print('\n-----------------------------------\n')
    print(Fore.GREEN, f"Agent: {result.output.explanation}")
    color = Fore.GREEN if result.output.decision_type == "Approved" else Fore.RED
    print(color, f"Decision: {result.output.decision_type}")
    print(color, f"Covered amount: ${result.output.covered_amount}")
    print(color, f"Patient responsibility: ${result.output.patient_responsibility}")
    print(color, f"Authorization code: {result.output.authorization_code}")
    print('\n-----------------------------------\n')

except ModelRetry as e:
    print(Fore.RED, e)
except Exception as e:
    print(Fore.RED, e)

In [ ]:
# Run the agent
# Run the agent
try:
    # Create a patient profile and insurance policy
    patient = Patient(
        patient_id=78901,
        name="Robert Miller",
        email="robert.m@example.com",
        age=52,
        insurance_id="INS456789",
        diagnosis_code="M54.5",  # Low back pain
        procedure_code="97140",  # Manual therapy techniques
        treatment_cost=1200,
        treatment_description="Chiropractic adjustment and massage therapy for chronic low back pain",
        provider_in_network=False,  # Out-of-network provider
        treatment_urgency="Elective",
        previous_treatments="Over-the-counter pain medication, heat therapy",
        medical_history="Chronic low back pain (3 years), Hypertension",
        additional_notes="Patient seeking alternative treatment after minimal relief from conventional methods"
    )

    policy = InsurancePolicy(
        policy_number=34567,
        policy_type="HMO",  # HMO typically requires in-network providers
        policy_status="Active",
        annual_deductible=2000,
        out_of_pocket_max=6000,
        remaining_deductible=1800,  # Most of deductible not yet met
        requires_preauthorization=True,
        excluded_procedures=["97140", "98940", "98941"]  # Excluded chiropractic procedures
    )

    # Create dependencies
    deps = Deps(patient=patient, policy=policy)

    question = "Please evaluate this treatment authorization request."
    result = agent.run_sync(question, deps=deps)

    print('\n-----------------------------------\n')
    print(Fore.GREEN, f"Agent: {result.output.explanation}")
    color = Fore.GREEN if result.output.decision_type == "Approved" else Fore.RED
    print(color, f"Decision: {result.output.decision_type}")
    print(color, f"Covered amount: ${result.output.covered_amount}")
    print(color, f"Patient responsibility: ${result.output.patient_responsibility}")
    print(color, f"Authorization code: {result.output.authorization_code}")
    print(Fore.YELLOW, f"Appeal options: {result.output.appeal_options}")
    print('\n-----------------------------------\n')

except ModelRetry as e:
    print(Fore.RED, e)
except Exception as e:
    print(Fore.RED, e)